In [4]:
import os 
import os.path as osp
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import random
from collections import Counter
import open3d as o3d
import open3d.core as o3c
import json
import subprocess

import pickle
from  plyfile import PlyData
import torch
from tqdm import tqdm
import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r,visualisation


#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
frame_number = "000015"
img_width = 960
img_height = 540

curr_scan_id = "38770c9d-86d7-27b8-869e-4f713b04f290" #is ref 1d2f8510-d757-207c-8c48-3684433860e1
new_scan_id =  "38770c95-86d7-27b8-8717-3485b411ddc7" #is rescan 9c27de56-6184-2cda-8196-591957b6387d

#the original meshes are given in the file  'labels.instances.annotated.v2.ply'



/local/home/ekoller/BT


compute the object centers

In [25]:
#access the data of the object centers
curr_scene_path = osp.join(data_dir, "files", "orig", "data", curr_scan_id + ".pkl")
with open(curr_scene_path, 'rb') as f:
    data = pickle.load(f)

obj_ids_pkl = data["objects_id"]


#access the things for the mesh
pathToMesh = osp.join(data_dir,"scenes", scan_id, "labels.instances.align.annotated.v2.ply")
ply_data = PlyData.read(pathToMesh)
vertices = ply_data['vertex'].data
vertex_array = np.array([list(vertex) for vertex in vertices])

# Extract x, y, z coordinates and objectId
x = vertex_array[:, 0]
y = vertex_array[:, 1]
z = vertex_array[:, 2]
object_ids_mesh = vertex_array[:, 6]  # Assuming 'objectId' is the 7th property

unique_object_ids = np.unique(object_ids_mesh)

bounding_boxes_tmp = {}
centroids = {}
#go over every id and compute the box
for obj_id in unique_object_ids:
                #we want the same ids for the boxes
                if obj_id in obj_ids_pkl:
                        # Filter vertices by object ID
                        obj_mask = object_ids_mesh == obj_id
                        obj_coords = np.vstack((x[obj_mask], y[obj_mask], z[obj_mask])).T
                        
                        #also compute the centroid
                        centroid = np.mean(obj_coords, axis=0)
                        centroids[obj_id] = centroid

print(centroids)

{1.0: array([-0.8052995 , -0.09936274, -1.3703212 ], dtype=float32), 2.0: array([-1.2946086,  1.306983 , -0.9697552], dtype=float32), 3.0: array([-0.15803622,  1.1367626 , -1.2699404 ], dtype=float32), 4.0: array([-0.67319775,  1.946421  , -0.0800736 ], dtype=float32), 5.0: array([-2.7239525 ,  0.45978028,  0.09460288], dtype=float32), 6.0: array([-2.7053003 ,  0.1962266 , -0.15341012], dtype=float32), 7.0: array([-3.0109458 ,  0.5486571 ,  0.00872199], dtype=float32), 8.0: array([-1.038778  , -1.3289794 , -0.20251158], dtype=float32), 9.0: array([-2.2459922, -0.9997788, -0.0482852], dtype=float32), 10.0: array([-1.6873078 ,  2.2443333 , -0.09623797], dtype=float32), 11.0: array([ 1.4278423 , -1.4321533 , -0.44935113], dtype=float32), 12.0: array([ 1.6850723 , -0.5469698 ,  0.25493035], dtype=float32), 13.0: array([ 1.4222167 ,  1.0109353 , -0.15785623], dtype=float32), 14.0: array([-0.4559756,  0.1674362,  1.0433657], dtype=float32), 15.0: array([-2.1124775 , -0.7270743 , -0.81892896]

this segment gets for an object id the colour the object is assigned

In [28]:
#for a given scene get the colours of the differnt object_ids as a dictionary
def get_id_colours(data_dir,scan_id):
    #access the mesh file to get the colour of the ids
    mesh_file = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
    ply_data = PlyData.read(mesh_file)
    # Extract vertex data
    vertices = ply_data['vertex']
    vertex_count = len(vertices)
    
    # Initialize dictionary to store object_id -> color mappings
    object_colors = {}
    
   # Iterate through vertices
    for i in range(vertex_count):
        vertex = vertices[i]
        object_id = vertex['objectId']
        color = (vertex['red'], vertex['green'], vertex['blue'])
        
        # Check if object_id already in dictionary, otherwise initialize a Counter
        if object_id in object_colors:
            object_colors[object_id][color] += 1
        else:
            object_colors[object_id] = Counter({color: 1})
    
    # Convert Counter to dictionary with most frequent color
    for object_id, color_counter in object_colors.items():
        most_common_color = color_counter.most_common(1)[0][0]
        object_colors[object_id] = np.array(most_common_color)
    
    return object_colors

access the data of the object centers

In [ ]:
#access the data of the object centers
curr_scene_path = osp.join(data_dir, "files", "orig", "data", curr_scan_id + ".pkl")
with open(curr_scene_path, 'rb') as f:
    data = pickle.load(f)

# Check if the loaded data is a dictionary
if isinstance(data, dict):
    print("Keys in the pkl file:")
    for key in data.keys():
        print(key)
else:
    print("The loaded data is not a dictionary. It is of type:", type(data))

this codesegment is here to make sure that the data is alligned before accessed + it visualizes the points based on npy file

In [32]:


def load_point_cloud_from_npy(npy_file_path):
    # Load the .npy file
    data = np.load(npy_file_path, allow_pickle=True)

    # Extract the fields (x, y, z, and RGB)
    x = data['x']
    y = data['y']
    z = data['z']

    # Extract the RGB color information (optional)
    r = data['red']
    g = data['green']
    b = data['blue']

    # Create a numpy array of the 3D coordinates
    points = np.column_stack((x, y, z))  # Combine x, y, z into an array of points

    # Create a numpy array of the RGB colors
    colors = np.column_stack((r, g, b)) / 255.0  # Normalize to [0, 1]
    

    # Create an Open3D point cloud object
    pcd = o3d.geometry.PointCloud()

    # Set the points and colors to the point cloud
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    return pcd

def create_centroid_point_cloud(centroids, color=[1, 0, 0], radius=0.1):
    # Create a new point cloud for centroids
    centroid_pcd = o3d.geometry.PointCloud()
    color_dict = get_id_colours(data_dir, curr_scan_id)
    # Convert centroids to numpy array if not already
    centroid_points = np.array(list(centroids.values()))

    # Create a point cloud for centroids
    centroid_pcd.points = o3d.utility.Vector3dVector(centroid_points)
     # Initialize list to store colors
    centroid_colors = []

    for obj_id in centroids.keys():
        # Fetch color for the current object ID from color_dict
        color = color_dict.get(obj_id, [1.0, 1.0, 1.0])  # Default to white if not found
        normalized_color = np.array(color) / 255.0
        normalized_color = normalized_color*0.5
        centroid_colors.append(normalized_color)
    
    centroid_pcd.colors = o3d.utility.Vector3dVector(centroid_colors)

    # Create spheres to visualize centroids as larger blobs
    spheres = []
    for point, color in zip(centroid_points, centroid_colors):
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
        sphere.translate(point)
        sphere.paint_uniform_color(color)
        spheres.append(sphere)

    # # Set a uniform color for all centroids
    # centroid_pcd.colors = o3d.utility.Vector3dVector([color] * len(centroid_points))

    # # Create spheres to visualize centroids as larger blobs
    # spheres = []
    # for point in centroid_points:
    #     sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    #     sphere.translate(point)
    #     sphere.paint_uniform_color(color)
    #     spheres.append(sphere)
    
    return centroid_pcd, spheres


data_ref =osp.join(scenes_dir, curr_scan_id, "data.npy")
ref_pointcloud = load_point_cloud_from_npy(data_ref)

data_rescan = osp.join(scenes_dir, new_scan_id, "data.npy")
rescan_pointcloud = load_point_cloud_from_npy(data_rescan)

centroid_pcd, centroid_spheres = create_centroid_point_cloud(centroids, color=[1, 0, 0], radius=0.1)


# Visualize both point clouds together
o3d.visualization.draw_geometries([ref_pointcloud, rescan_pointcloud] + centroid_spheres, window_name="Combined Point Clouds", width=800, height=600)





this segment displays the meshes of the scans based on the ply file

In [3]:
#sanity check if the transformations overall actually do what they are supposed to do using visuals

"""
curr_scan_:id mesh displayed in the reference frame
"""
pathToMesh = osp.join(data_dir,"scenes", curr_scan_id, "labels.instances.annotated.v2.ply")
curr_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not curr_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(curr_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(curr_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
curr_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)



"""
New_scan_id mesh display in original coordinate system (rescan coord)
"""
pathToMesh = osp.join(data_dir,"scenes", new_scan_id, "labels.instances.annotated.v2.ply")
new_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not new_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(new_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(new_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)


"""
Display everything
"""

##visualize everything
vis = o3d.visualization.Visualizer()
vis.create_window(width=800, height=600)

vis.add_geometry(curr_mesh)

vis.add_geometry(new_mesh)



vis.run()
vis.destroy_window()

for the object centers we use the mesh to calculate it 

In [19]:
plydata_npy = np.load(osp.join(scenes_dir, curr_scan_id, "data.npy"), allow_pickle=True)

#print the names of the datafields
if plydata_npy.dtype.names:
        for name in plydata_npy.dtype.names:
                print(f"Field: {name}")
                print(plydata_npy[name])
else:
    print(plydata_npy)


Field: x
[0.13    0.09    0.05    ... 1.37448 1.39    1.38084]
Field: y
[-1.15 -1.15 -1.15 ... -0.47 -0.49 -0.45]
Field: z
[ 1.04296   1.04576   1.04274  ... -0.41     -0.361669 -0.35    ]
Field: red
[227 227 227 ... 146 146 146]
Field: green
[119 119 119 ... 111 111 111]
Field: blue
[194 194 194 ... 194 194 194]
Field: objectId
[14 14 14 ... 34 34 34]
Field: globalId
[ 82  82  82 ... 476 476 476]
Field: NYU40
[22 22 22 ... 27 27 27]
Field: Eigen13
[3 3 3 ... 7 7 7]
Field: RIO27
[15 15 15 ... 18 18 18]
